In [3]:
import pandas as pd

df = pd.read_excel("../data/raw/GSAF5.xls")

## Cleaning the dataset

In [4]:
#Cleaning up 'activity' column in the dataset

df['Activity'].nunique() #unique values in the activity column

1608

In [5]:
df['activity_clean'] = (
    df['Activity']
    .str.lower() #lowercase
    .str.strip() #remove spaces
    .str.replace(r'[^a-z\s]', '', regex=True)  #remove punctuation/numbers
)

In [6]:
import numpy as np

def categorize_activity(text):
    if pd.isna(text): #handling 'NaN' values and labelling them as unknown
        return "Unknown"
    if any(word in text for word in ["surf", "bodyboard", "paddle", "boogie", "body boarding", "kiteboarding", "foilboarding", "skimboarding", "wakeboarding"]): #if the word is present in the text, then loop over each word and replace with return 'string'
        return "Surfing"
    elif any(word in text for word in ["swim", "bathing", "snorkel", "rescue", "float", "splash", "swimming"]):
        return "Swimming"
    elif any(word in text for word in ["fish", "spearfish", "net", "catch", "line", "fishing", "spear", "scalloping", "lobstering", "hunt", "clamming"]):
        return "Fishing"
    elif any(word in text for word in ["dive", "scuba", "freediv", "underwater", "research", "investigat", "pearl", "recover", "diving"]): #'investigat' covers terms like investigating / investigation / investigator..
        return "Diving"
    elif any(word in text for word in ["boat", "kayak", "sail", "ship", "vessel", "frigate", "dinghy", "canoe", "race", "compet", "rowing", "watercraft", "jet ski", "paddling", "sculling", "raft", "yacht"]):
        return "Boating"
    elif any(word in text for word in ["walk", "stand", "wade", "reef", "shore", "beach", "adrift", "wading", "tread"]):
        return "Wading"
    elif any(word in text for word in ["sea disaster", "aircraft", "boeing", "wreck", "hurricane", "tsunami", "earthquake", "disaster", "plunged", "sank", "destroyed", "overboard", "suicide", "air", "petting", "capsize", "swept", "help", "ride", "sunk", "went down", "crash", "sinking"]):
        return "Catastrophe"
    else:
        return "Other Activity"

df['Activity_group'] = df['activity_clean'].apply(categorize_activity) #new column 'activity group' for categorised activity


In [7]:
print("Original:", df['Activity'].nunique()) #original dataset
print("Cleaned:", df['activity_clean'].nunique()) #activity column after removing spaces, fixing inconsistent letter cases and removing punctutations
print("Grouped:", df['Activity_group'].nunique()) #after categorising all the activity under specific keywords

Original: 1608
Cleaned: 1522
Grouped: 9


In [8]:
df[df['Activity_group'] == 'Other Activity']['activity_clean'].sample(40, random_state=42).tolist()
#sample of 'other' category to improve mapping and grouping

['baiting sharks',
 'unknown but it was said to be the first known attack in sydney harbour',
 'attempting to fix motor',
 'sitting in shallows',
 'sitting in the water',
 'washing cooking pans',
 'human head found in shark caught by british steamer syria',
 'playing in the water',
 'pulling anchor',
 'escaping from alacatraz',
 'measuring sharks',
 'filming',
 'washing his feet',
 'stamding',
 'masted steel barque glenbank foundered during a cyclone',
 'filming  blue shark',
 'lying prone in  of water',
 'waterskiing',
 'kite boarding',
 'dangling feet in the water',
 'jumped into the water',
 'attempting to lasso sharks tail',
 'hms victoria collided with the hms camperdown',
 'escaping from alacatraz',
 'on a roundtheworld expedition',
 'attempting to illegally enter the usa',
 'inebriated woke from sleep and fell off deck into the water',
 'fell into the water',
 'sitting in shallows',
 'hbm magpie foundered in a squall',
 'feeding mullet to sharks',
 'parachuted from balloon',
 't

In [9]:
df['Activity_group'].value_counts()

Activity_group
Swimming          1667
Surfing           1637
Fishing           1368
Unknown            585
Diving             568
Wading             387
Boating            332
Other Activity     332
Catastrophe        174
Name: count, dtype: int64

In [10]:
#Cleaning and formatting the 'date' column and extracting 'month' from it
df["Date"].head(30)

0            14th October
1            11th October
2             7th October
3          29th September
4          27th September
5           6th September
6           1st September
7             30th August
8             18th August
9             17th August
10            16th August
11             7th August
12             1st August
13              28th July
14              25th July
15              22nd July
16              20th July
17              19th July
18              18th July
19              15th July
20              6th July 
21               6th July
22               4th July
23              29th June
24              25th June
25              22nd June
26              17th June
27    2025-06-11 00:00:00
28               31st May
29    2025-05-29 00:00:00
Name: Date, dtype: object

In [11]:
df["Date"].tail(30)

7020                                         World War II
7021                                          Before 1905
7022                              A few years before 1938
7023                                              No date
7024                                          Early 1930s
7025                                          Before 1927
7026                                  Between 1918 & 1939
7027                                              No date
7028                                              No date
7029                                              No date
7030                                           1920 -1923
7031                                          Before 1921
7032                                          Before 1911
7033                                          Before 1921
7034                                          Before 1921
7035                                          Before 1917
7036                                   Before 17-Jul-1916
7037    No dat

In [12]:
df['Date'] = df['Date'].astype(str)

In [13]:
import numpy as np

def extract_month_or_nan(value):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    value_str = str(value)
    for month in months:
        if month in value_str:
            return month
    return np.nan  #assign NaN if no month found

df['Extracted_Month_NaN'] = df['Date'].apply(extract_month_or_nan)

count_months = df['Extracted_Month_NaN'].count() #count extracted months (non-null values)
print(f'Number of extracted months: {count_months}')

count_nan = df['Extracted_Month_NaN'].isna().sum() #count how many NaN values were assigned
print(f'Number of non-month values: {count_nan}')

Number of extracted months: 6449
Number of non-month values: 601


In [14]:
df['Extracted_Month_NaN'].value_counts()

Extracted_Month_NaN
Jul    784
Aug    673
Sep    611
Jan    566
Jun    549
Oct    503
Apr    493
Dec    491
Mar    458
May    452
Nov    451
Feb    418
Name: count, dtype: int64

In [15]:
def get_season(month):
    if month in ["Dec", "Jan", "Feb"]:
        return "Winter"
    elif month in ["Mar", "Apr", "May"]:
        return "Spring"
    elif month in ["Jun", "Jul", "Aug"]:
        return "Summer"
    elif month in ["Sep", "Oct", "Nov"]:
        return "Autumn"
    else:
        return np.nan

df["Season"] = df["Extracted_Month_NaN"].apply(get_season) #applying the season assigning as a column to the dataframe

In [16]:
df['Extracted_Month_NaN'].value_counts()

Extracted_Month_NaN
Jul    784
Aug    673
Sep    611
Jan    566
Jun    549
Oct    503
Apr    493
Dec    491
Mar    458
May    452
Nov    451
Feb    418
Name: count, dtype: int64

In [17]:
cleaned_df = df

In [18]:
cleaned_df.to_csv("data/cleaned_attacks.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'data'

## Summary Statistics

In [19]:
#summary for categorical variables
df[['Activity_group', 'Season']].describe()


,Activity_group,Season
count,7050,6449
unique,9,4
top,Swimming,Summer
freq,1667,2006


- **Swimming** is the top most activity, that's associated with shark attacks
- **Summer** is the season where most of the shark attacks happen

In [20]:
#summary for categorical variable - 'Activity_group'
df['Activity_group'].value_counts()

Activity_group
Swimming          1667
Surfing           1637
Fishing           1368
Unknown            585
Diving             568
Wading             387
Boating            332
Other Activity     332
Catastrophe        174
Name: count, dtype: int64

In [21]:
#summary for categorical variable - 'Season'
df['Season'].value_counts()

Season
Summer    2006
Autumn    1565
Winter    1475
Spring    1403
Name: count, dtype: int64

## Visualizations

### Univariate Analysis

In [1]:
import plotly.express as px

#### 1. Activity groups related to shark attacks

In [22]:
import plotly.express as px

# Count the number of attacks per activity group
activity_counts = df['Activity_group'].value_counts().reset_index()
activity_counts.columns = ['Activity Group', 'Number of Attacks']

# Create a horizontal bar chart
fig = px.bar(activity_counts,
             x='Number of Attacks',
             y='Activity Group',
             color='Activity Group',
             orientation='h',  # make it horizontal
             title='Number of Shark Attacks by Activity',
             text='Number of Attacks')

# Adjust text and layout
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Number of Attacks',
                  yaxis_title='Activity Group',
                  showlegend=False,
                  margin=dict(t=100, b=40, l=100, r=40))
fig.show()


#### 2. Seasons corresponding to shark attacks

In [23]:
#Number of attacks per season
season_counts = df['Season'].value_counts().reset_index()
season_counts.columns = ['Season', 'Number of Attacks']

#create an interactive horizontal bar chart
fig = px.bar(season_counts,
             x='Number of Attacks',
             y='Season',
             color='Season',
             orientation='h',
             title='Number of Shark Attacks by Season',
             text='Number of Attacks')

#Layout adjustments
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Number of Attacks',
                  yaxis_title='Season',
                  showlegend=False,
                  margin=dict(t=100, b=40, l=60, r=40))
fig.show()

### Bivariate Analysis

#### 1. Relationship between activity group and victim fatality

In [ ]:
df["Fatal Y/N"].unique()

In [24]:
#Fatal Y/N column cleaning

df['Fatal Y/N'] = df['Fatal Y/N'].astype(str).str.strip().str.upper()

df['Fatal Y/N'] = df['Fatal Y/N'].replace({
    'UNKNOWN': 'N',  
    'F': 'N',
    'M': 'N',
    'NAN': 'N',
    'NQ': 'N',
    '2017': 'N',
    'Y X 2': 'Y',
})

In [ ]:
df["Fatal Y/N"].value_counts()

In [25]:
df['Fatal Y/N'] = df['Fatal Y/N'].replace({
    'N': 'Survived',  
    'Y': 'Did not survive'})

In [ ]:
df["Fatal Y/N"].value_counts()

In [26]:
import plotly.express as px
import pandas as pd

# Calculate proportions
activity_fatal = (
    df.groupby("Activity_group")["Fatal Y/N"]
    .value_counts(normalize=True)
    .rename("Proportion")
    .reset_index()
)

# Extract only fatal proportions for sorting
fatal_order = (
    activity_fatal[activity_fatal["Fatal Y/N"] == "Did not survive"]
    .sort_values("Proportion", ascending=False)["Activity_group"]
)

# Create the horizontal stacked bar chart, sorted by fatality proportion
fig = px.bar(
    activity_fatal,
    y="Activity_group", 
    x="Proportion",
    color="Fatal Y/N",
    barmode="stack",
    category_orders={"Activity_group": fatal_order.tolist()},
    text=activity_fatal["Proportion"].apply(lambda x: f"{x:.0%}"),
    title="Proportion of Fatal vs Survived Incidents by Activity Group (Sorted by Fatality Rate)",
    labels={
        "Activity_group": "Activity Group",
        "Proportion": "Proportion",
        "Fatal Y/N": "Outcome",
    },
    orientation='h'
)

# Layout adjustments
fig.update_traces(textposition="inside")
fig.update_layout(
    yaxis_tickangle=0, 
    xaxis_tickformat=".0%",
    legend_title="Fatality Outcome",
    bargap=0.25,
)

fig.show()


- About 65% of the victims of **Catastrophes** like aircraft accidents over seas, drowning, Tsunamis, Floods and Hurricanes did not survive.
- **Boating** is activity that resulted in the second most fatality.
- **Surfing** is the activity that resulted in the least amount of fatal incidents.

#### 2. Relationship between Activity group and Season of the attack

In [27]:
import pandas as pd 
import plotly.express as px 
#calculate total attack counts per season 
season_order_df = df.groupby('Season').size().reset_index(name='Total_Attacks') 
season_order = season_order_df.sort_values(by='Total_Attacks', ascending=False)['Season'].tolist() 
heatmap_data = df.groupby(['Season', 'Activity_group']).size().reset_index(name='Count')

fig = px.density_heatmap(
    heatmap_data,
    x='Season',
    y='Activity_group',
    z='Count',
    category_orders={'Season': season_order},
    color_continuous_scale='Blues',
    title='Activities related to shark attack across Seasons',
    labels={'Activity_group': 'Activity', 'Season': 'Season', 'Count': 'Number of Attacks'}
)

# Add annotations
for i, row in heatmap_data.iterrows():
    fig.add_annotation(
        x=row['Season'],
        y=row['Activity_group'],
        text=str(int(row['Count'])),
        showarrow=False,
        font=dict(color='black', size=10)
    )

fig.show()

- **Swimming** is the activity that resulted in the most shark attacks during **Summer**
- **Surfing** is activity that resulted in the most shark attacks during **Autumn**

In [ ]:
pip install meteostat

In [28]:
import re

# Fill missing values
df['Location'] = df['Location'].fillna('Unknown')
# Replace empty states with Unknown
df['State'] = df['State'].replace('', 'Unknown')


# Regex pattern to remove numbers and units
pattern = r'\b(\d+|miles|mi|km|kilometers)\b'

unwanted_words = ['miles', 'mi', 'km', 'kilometers', 'islands', 'near','nm', 'off', 
                  'below', 'the', 'of', 'east of', 'west of', 'south of',
                   'between', '&' , 'north of', 'ºN', 'ºS', 'ºW', 'ºE']                    
pattern = r'\b(?:' + '|'.join(unwanted_words) + r'|\d+)\b'

# Clean the Location column
df['location_clean'] = (
    df['Location']
    .str.replace(pattern, '', regex=True, flags=re.IGNORECASE)  # remove numbers & units
    .str.replace(r'\s+', ' ', regex=True)  # remove extra spaces
    .str.strip()
)

df['state_clean'] = (
    df['State']
    .str.replace(pattern, '', regex=True, flags=re.IGNORECASE)  # remove numbers & units
    .str.replace(r'\s+', ' ', regex=True)  # remove extra spaces
    .str.strip()
)
df['location_clean'] = df['location_clean'].replace('', 'Unknown').fillna('Unknown')

df['state_clean'] = df['state_clean'].replace('', 'Unknown').fillna('Unknown')


# Print the result
print(df[['Location', 'State', 'location_clean', 'state_clean']])

                                 Location                  State  \
0                       Catagena Province   Bolivar, del Isolate   
1          Cook Esplanade Thursday Island             Queensland   
2                         Kangaroo Island        South Australia   
3                         Catalina Island         Off California   
4                           Cocos Islands                    NaN   
...                                   ...                    ...   
7045                          Roebuck Bay      Western Australia   
7046                              Unknown      Western Australia   
7047                       Ocracoke Inlet         North Carolina   
7048                 Panama Bay 8ºN, 79ºW                    NaN   
7049  Below the English fort, Trincomalee       Eastern Province   

                      location_clean           state_clean  
0                  Catagena Province  Bolivar, del Isolate  
1     Cook Esplanade Thursday Island            Queensland  


In [29]:
# Subset the dataset to attacks from 2019 onwards
recent_attacks = df[(df['Year'] >= 2019) & (df['Year'] <= 2024)]

# Group by 'state_clean' and count attacks
top_states_recent = recent_attacks.groupby('state_clean').size().reset_index(name='Count')

# Sort descending by count and select top 5 states
top_5_states = top_states_recent.sort_values(by='Count', ascending=False).head(5)

print("Top 5 states by attacks in recent 5 years:")
print(top_5_states)

Top 5 states by attacks in recent 5 years:
           state_clean  Count
30             Florida    133
44              Hawaii     44
136  Western Australia     44
98          Queensland     32
75     New South Wales     30


In [30]:
from meteostat import Stations, Monthly
import pandas as pd
from datetime import datetime

#mapping states to approximate coordinates
state_coords = {
    'Florida': (27.994402, -81.760254),
    'Western Australia': (-31.9505, 115.8605), #coordinates for Western Australia, used Perth approximately
    'Hawaii': (21.3069, -157.8583),
    'Queensland': (-20.9176, 142.7028),
    'New South Wales': (-31.2532, 146.9211),   
}

start = datetime(2019, 1, 1)
end = datetime(2024, 12, 31)

def fetch_yearly_avg_temp(name, lat, lon):
    stations = Stations().nearby(lat, lon).fetch(1)
    if stations.empty:
        print(f"No station found for {name}")
        return None
    station_id = stations.index[0]
    data = Monthly(station_id, start, end).fetch()
    if 'tavg' not in data.columns:
        if 'tmin' in data.columns and 'tmax' in data.columns:
            data['tavg'] = (data['tmin'] + data['tmax']) / 2
        else:
            print(f"Average temperature data unavailable for {name}")
            return None
    data = data.reset_index()
    data['Year'] = data['time'].dt.year
    yearly_avg = data.groupby('Year')['tavg'].mean().reset_index()
    yearly_avg.rename(columns={'tavg': 'Avg_Temperature_C'}, inplace=True)
    yearly_avg['state_clean'] = name
    return yearly_avg

#fetching temperature data only for top 5 states found
top_states_list = top_5_states['state_clean'].tolist()

temp_dfs = []
for state in top_states_list:
    coords = state_coords.get(state)
    if coords:
        temp_data = fetch_yearly_avg_temp(state, coords[0], coords[1])
        if temp_data is not None:
            temp_dfs.append(temp_data)
    else:
        print(f"Coordinates for {state} not found. Temperature data not fetched.")

#to combine all temperature data into one DataFrame
if temp_dfs:
    temp_all = pd.concat(temp_dfs, ignore_index=True)
    print("Temperature data fetched for top states:")
    print(temp_all.head(30))
else:
    print("No temperature data was fetched.")

Temperature data fetched for top states:
    Year  Avg_Temperature_C        state_clean
0   2019               19.1            Florida
1   2020          23.333333            Florida
2   2021             22.825            Florida
3   2022          22.766667            Florida
4   2023               23.4            Florida
5   2024          22.809091            Florida
6   2019          26.258333             Hawaii
7   2020          26.058333             Hawaii
8   2021          25.741667             Hawaii
9   2022              25.75             Hawaii
10  2023               25.9             Hawaii
11  2024             25.675             Hawaii
12  2019          18.683333  Western Australia
13  2020               18.9  Western Australia
14  2021          18.641667  Western Australia
15  2022             18.675  Western Australia
16  2023          18.908333  Western Australia
17  2024              19.35  Western Australia
18  2019               24.8         Queensland
19  2020          2

In [31]:
#aggregating shark attack data by year and state for number of attacks
attack_counts = recent_attacks.groupby(['Year', 'state_clean']).size().reset_index(name='Attack_Count')

#merge on year and state_clean
merged_df = pd.merge(
    attack_counts,
    temp_all,
    on=['Year', 'state_clean'],
    how='left'
)

print("Merged dataset sample:")
print(merged_df.head(10))


Merged dataset sample:
     Year            state_clean  Attack_Count  Avg_Temperature_C
0  2019.0                  Abaco             2               <NA>
1  2019.0                   Baja             1               <NA>
2  2019.0                  Bélep             1               <NA>
3  2019.0             California             6               <NA>
4  2019.0                 Cayman             1               <NA>
5  2019.0       Central Province             1               <NA>
6  2019.0                Chatham             1               <NA>
7  2019.0  Eastern Cape Province             4               <NA>
8  2019.0    Fernando de Noronha             2               <NA>
9  2019.0                Florida            26               19.1


In [32]:

selected_states = ["Florida", "Western Australia", "Hawaii", "Queensland", "New South Wales"]
filtered_df = merged_df[merged_df['state_clean'].isin(selected_states)]


In [34]:
import plotly.graph_objects as go

# Filter only top 5 states and years 2019–2024
top_states = ["Florida", "Western Australia", "Hawaii", "Queensland", "New South Wales"]
filtered_df = merged_df[
    (merged_df['state_clean'].isin(top_states)) &
    (merged_df['Year'].between(2019, 2024))
]

# Assign distinct colours for each state
colors = {
    "Florida": "#1f77b4",          # blue
    "Western Australia": "#ff7f0e", # orange
    "Hawaii": "#2ca02c",           # green
    "Queensland": "#d62728",       # red
    "New South Wales": "#9467bd"   # purple
}

fig = go.Figure()

for state in top_states:
    state_data = filtered_df[filtered_df['state_clean'] == state]
    color = colors[state]

    # Solid line for attacks
    fig.add_trace(go.Scatter(
        x=state_data["Year"],
        y=state_data["Attack_Count"],
        name=f"{state} - Attacks",
        mode="lines+markers",
        line=dict(color=color, width=2),
        marker=dict(symbol="circle")
    ))

    # Dotted line for temperature (same colour)
    fig.add_trace(go.Scatter(
        x=state_data["Year"],
        y=state_data["Avg_Temperature_C"],
        name=f"{state} - Temperature (°C)",
        mode="lines+markers",
        line=dict(color=color, dash="dot", width=2),
        marker=dict(symbol="x")
    ))

fig.update_layout(
    title="Year-wise Shark Attacks and Average Temperature (2019–2024)",
    xaxis_title="Year",
    yaxis_title="Count / Avg Temperature (°C)",
    template="plotly_white",
    legend_title="Legend",
    width=950,
    height=550
)

fig.show()


In [35]:
import re

# Fill missing values
df['Location'] = df['Location'].fillna('Unknown')
# Replace empty states with Unknown
df['State'] = df['State'].replace('', 'Unknown')


# Regex pattern to remove numbers and units
pattern = r'\b(\d+|miles|mi|km|kilometers)\b'

unwanted_words = ['miles', 'mi', 'km', 'kilometers', 'islands', 'near','nm', 'off', 
                  'below', 'the', 'of', 'east of', 'west of', 'south of',
                   'between', '&' , 'north of', 'ºN', 'ºS', 'ºW', 'ºE']                    
pattern = r'\b(?:' + '|'.join(unwanted_words) + r'|\d+)\b'

# Clean the Location column
df['location_clean'] = (
    df['Location']
    .str.replace(pattern, '', regex=True, flags=re.IGNORECASE)  # remove numbers & units
    .str.replace(r'\s+', ' ', regex=True)  # remove extra spaces
    .str.strip()
)

df['state_clean'] = (
    df['State']
    .str.replace(pattern, '', regex=True, flags=re.IGNORECASE)  # remove numbers & units
    .str.replace(r'\s+', ' ', regex=True)  # remove extra spaces
    .str.strip()
)
df['location_clean'] = df['location_clean'].replace('', 'Unknown').fillna('Unknown')

df['state_clean'] = df['state_clean'].replace('', 'Unknown').fillna('Unknown')


# Print the result
print(df[['Location', 'State', 'location_clean', 'state_clean']])

                                 Location                  State  \
0                       Catagena Province   Bolivar, del Isolate   
1          Cook Esplanade Thursday Island             Queensland   
2                         Kangaroo Island        South Australia   
3                         Catalina Island         Off California   
4                           Cocos Islands                    NaN   
...                                   ...                    ...   
7045                          Roebuck Bay      Western Australia   
7046                              Unknown      Western Australia   
7047                       Ocracoke Inlet         North Carolina   
7048                 Panama Bay 8ºN, 79ºW                    NaN   
7049  Below the English fort, Trincomalee       Eastern Province   

                      location_clean           state_clean  
0                  Catagena Province  Bolivar, del Isolate  
1     Cook Esplanade Thursday Island            Queensland  


In [36]:
# Plot as horizontal bar chart
fig = px.bar(
    fatality_location,
    y='location_clean',  # y-axis for horizontal bars
    x='Count',           # x-axis for length of bars
    color='Fatal Y/N',
    barmode='group',
    orientation='h',  # Horizontal bars
    title='Fatal vs Non-Fatal Shark Attacks by Location',
    labels={'location_clean': 'Location', 'Count': 'Number of Attacks', 'Fatal Y/N': 'Outcome'},
    color_discrete_map={'Did not survive': '#d62728', 'Survived': '#2ca02c'},
    text='Count'
)

# Position the text outside bars (for horizontal, at right of bars)
fig.update_traces(textposition='outside')

# Update layout so axes are correctly labeled
fig.update_layout(
    yaxis=dict(title='Location'),
    xaxis=dict(title='Number of Attacks', range=[0, fatality_location['Count'].max() * 1.15]),
    plot_bgcolor='white',
    title_x=0.5,
    legend_title_text='Outcome'
)

fig.show()


NameError: name 'fatality_location' is not defined